In [1]:
import cv2
import numpy as np
import os

import matplotlib.pyplot as plt

from skimage.feature import canny

import scipy.ndimage as nd

from skimage.filters import sobel
from skimage.segmentation import watershed

In [6]:


image_formats = ["jpg", 'jpeg', 'png']
file_list = os.listdir('./input')


In [7]:
def edge_to_image(edges):
	ls = []
	for i in range(edges.shape[0]):
		ls.append([0])
		for j in range(edges.shape[1]):
			filters = 255 if edges[i][j] else 0
			ls[i].append(filters)
	return np.matrix(ls, dtype=np.float32)


def k_means_(img):
    Z = img.reshape((-1, 2))
    Z = np.float32(Z)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    K = 3
    ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))
    cv2.imshow('Clustering: K-Means Segmentation', res2)
    cv2.waitKey(0)   


def image_segmentation(image_name, dir_path='./input'):
    image_path = f'{dir_path}/{image_name}'
    image1 = cv2.imread(image_path)
    img = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)


    ret, thresh1 = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY +
                                 cv2.THRESH_OTSU)
    cv2.imshow('Threshold: Otsu+binary', thresh1)

    cv2.waitKey(0)

    thresh2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 199, 5)
    cv2.imshow('Threshold: adaptive', thresh2)
    cv2.waitKey(0)
    
    edges = canny(img)
    
    edge_image = edge_to_image(edges)
    
    cv2.imshow('Edge: Edge detector',edge_image)
    cv2.waitKey(0)
 
    fill_im = nd.binary_fill_holes(edges)
    fill_image = edge_to_image(fill_im)
    cv2.imshow('Region: Region Filling', fill_image)
    cv2.waitKey(0)

    elevation_map = sobel(img)
    
    fill_image = elevation_map*10
    cv2.imshow('Watershed: Elevation Map', fill_image)
    cv2.waitKey(0)

    markers = np.zeros_like(img)
    
    markers[img < 80] = 1
    markers[img > 150] = 10

    segmentation = watershed(elevation_map, markers)    
    cv2.imshow('Watershed: Watershed segmentation', np.matrix(segmentation/10, dtype = np.float32))
    cv2.waitKey(0)

    k_means_(img)
    
    cv2.destroyAllWindows()